In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\kisho\\Desktop\\PodcastListeningTimePrediction\\PodcastListeningTimePrediction'

In [3]:
os.chdir("../")

In [3]:
%pwd

'C:\\Users\\kisho\\Desktop\\PodcastListeningTimePrediction\\PodcastListeningTimePrediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    zip_file_path: Path
    unzip_dir: Path

In [5]:
from PodcastListeningTimePrediction.constants import *
from PodcastListeningTimePrediction.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            schema_file_path=SCHEMA_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        root_dir = self.config.data_ingestion.root_dir
        zip_file_path = self.config.data_ingestion.zip_file_path
        unzip_dir = self.config.data_ingestion.unzip_dir

        create_directories([Path(root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(root_dir),
            zip_file_path=Path(zip_file_path),
            unzip_dir=Path(unzip_dir)
        )
        return data_ingestion_config


In [6]:
from PodcastListeningTimePrediction import logger
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def extract_zip(self) -> None:
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        with ZipFile(self.config.zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  
        logger.info(f"Data unzipped to {self.config.unzip_dir}")

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_zip()
except Exception as e:
    logger.exception(f"An error occurred during data ingestion: {e}")
    raise e

[2025-07-22 18:50:12,000] - INFO:  YAML file config\config.yaml loaded successfully.
[2025-07-22 18:50:12,011] - INFO:  YAML file schema.yaml loaded successfully.
[2025-07-22 18:50:12,028] - INFO:  YAML file params.yaml loaded successfully.
[2025-07-22 18:50:12,034] - INFO:  Created directory: artifacts\data_ingestion
[2025-07-22 18:50:12,177] - INFO:  Data unzipped to artifacts\data_ingestion
